<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L05_1_User_free_Models_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!unzip hetrec2011-lastfm-2k.zip

--2024-04-13 01:46:15--  https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  10.1MB/s    in 0.2s    

2024-04-13 01:46:16 (10.1 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


In [ ]:
artists = {}

with open('artists.dat', 'r') as f:
    print(f.readline())
    for line in f:
        id, name, _, _ = line.split('\t')
        artists[int(id)] = name

id	name	url	pictureURL



In [ ]:
!head user_artists.dat

userID	artistID	weight
2	51	13883
2	52	11690
2	53	11351
2	54	10300
2	55	8983
2	56	6152
2	57	5955
2	58	4616
2	59	4337


In [ ]:
import numpy as np

users = []
items = []
counts = []

with open('user_artists.dat', 'r') as f:
    print(f.readline())
    for line in f:
        uid, mid, count = line.split('\t')

        users.append(int(uid))
        items.append(int(mid))
        counts.append(np.log(1+float(count)))

users = np.array(users, dtype=np.int32)
items = np.array(items, dtype=np.int32)
counts = np.array(counts)

userID	artistID	weight



In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
from scipy.sparse import csc_matrix
from tqdm import tqdm

n_items = max(items) + 1
train = csc_matrix((counts, (users, items)))
model = ElasticNet(alpha=0.1, l1_ratio=0.5, positive=True, fit_intercept=False, copy_X=False, selection='random', tol=1e-6, max_iter=100)

rows, cols, data = [], [], []
for current_item in tqdm(range(n_items)):

    y = train[:,current_item].toarray()
    if all(y==0): continue

    s = train.indptr[current_item]
    e = train.indptr[current_item+1]
    backup = train.data[s:e].copy()
    train.data[s:e] = 0.0

    model.fit(train, y)

    rows.extend(model.sparse_coef_.indices)
    cols.extend([current_item] * model.sparse_coef_.getnnz())
    data.extend(model.sparse_coef_.data)

    train.data[s:e] = backup

W = csc_matrix((data, (rows, cols)), shape=(n_items, n_items))

100%|██████████| 18746/18746 [03:36<00:00, 86.47it/s] 


In [ ]:
#1918 동방신기
#2117 시크릿
#2094 BoA
#1904 SHINee
#227 The Beatles

q = 1918

s = W.indptr[q]
e = W.indptr[q+1]
scores = W.data[s:e]
ids = W.indices[s:e]
ids_sorted = ids[scores.argsort()[::-1]]

print(f"[{artists[q]}]와 유사한 artists:")

for id in ids_sorted:
    print(f"- {artists[id]}")


NameError: name 'W' is not defined

In [ ]:
# 312 한국인
# 43 외국인?
uid = 312
user_counts = train[uid,:].tocsr()
scores = user_counts.dot(W)
order = scores.data.argsort()[::-1]

k = 10

for o in order:
    artist_id = scores.indices[o]
    score = scores.data[o]
    if artist_id not in user_counts.indices:
        print(artist_id, artists[artist_id], score)
        k -= 1

    if k == 0: break